In [1]:
"""
Created on Tue Jul 21 12:27:53 2020

@author: 97250
"""

path =r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\YouTube_USvideos.csv'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
data =  pd.read_csv(path)

In [2]:
# agg count function
def agg(colum,k ='',c='likes'):

    """
    Aggregation function.

    Input:
        Colum - string or list of strings.
        c - string, kind of column to pick.
        k - string, kind of agg. .

    Return:
        DataFrame's aggregated form by given column(s).
    """

    a = data.groupby(colum)

    # These function change due to the data kind I work with.
    if k == 'v':
        a = a.sum()
    else:
        a = a.count()

    if type(colum) != list: # if one column has been given
        return a[c].sort_values(ascending=False)
    else:
        return a[[c]].unstack(0)

In [3]:
#       partial data        ###

#grab 10% of data
##data = data.iloc[:round(0.1*len(data))]

# grab randomally
## much preffered
data= data.sample(frac =0.1, random_state =101)

# drop unneecessary columns
data.drop(['channel_title','description','thumbnail_link'] , axis=1,inplace=True)

################################

In [4]:
###         add columns        #####

#       mathmatical

# string to date
data['publish_time'] = data['publish_time'].apply(lambda x: pd.to_datetime(x.split('T')[0]))

## data['trending_date'] = data['trending_date'].apply(lambda x: pd.to_datetime(x.replace('.','-')))
## data['differ'] = data['trending_date'] - data['publish_time']

# drop all date columns ,keep differ and publish year and month
## data.drop(['publish_time','trending_date'],axis=1,inplace = True)

#month and year
data['month_publish'] = data['publish_time'].apply(lambda x: x.month)
data['year_publish']  = data['publish_time'].apply(lambda x: x.year )


# ratios values
data['like/dislike']  = data['likes']    /data['dislikes']
data['dislike/views'] = data['dislikes'] /data['views']


data.drop(['publish_time','trending_date'] ,axis=1,inplace=True)

### and keep all the other columns

##################

In [5]:
#           NLP



data['official'] = data['title'].apply(lambda x: 1 if 'official' in x.lower() else 0)

#question/abswer mark
data['answer_mark']   = data['title'].apply(lambda x : x.count('!'))
data['question_mark'] = data['title'].apply(lambda x : x.count('?'))

data['tagNums'] = data['tags'].apply(len)

data['capital'] = data['title'].apply(lambda x : sum(1 for c in x if c.isupper()))

import statistics 

#     harmonic mean of speciel sign in title

## The idea here is that combination of !,??,ASDEFGV   can show more
## of the title's message than all these signs in seperate
## And this is what harmonic AVG is about.
## Only if the title has all three signs than 'harmonic' > 0.

data['harmonic'] = data[['answer_mark','question_mark','capital']].apply(lambda x: statistics.harmonic_mean(x) ,axis=1)

###

# drop data
data.drop(['title','tags'],axis=1,inplace=True)


#################################################

In [6]:
#change to numerical data
data['video_error_or_removed'] = data['video_error_or_removed'].apply(lambda x:0 if False else 1)

#dealing with categorial variable - 'categorial_id'
dummies = pd.get_dummies(data['category_id'] , drop_first=True)

data = pd.concat([data.drop('category_id',axis=1),dummies],axis=1)

# drop rest of unnecessary columns
data.drop(['ratings_disabled','comments_disabled'],axis=1,inplace=True)


data.set_index('video_id',inplace =True)

In [7]:
#           NA


data.isna().sum()

## if likes and dislikes ==0 than their ratio is NAN
#lets fill in with 0

data.fillna(0,inplace=True)

In [8]:
"""
like/dislike

x/0 = meanningLess            Here is the problem
0/x = 0
0/0 = NAN # filled in
x/x = V
##
##

dislike/views'

x/0 = cant happen ,right?
0/x = 0 
0/0 = wasnot # anyway filled in with 0...
x/x = V

"""

infData = data[(data['likes']>0)&(data['dislikes']==0)]

data.drop(infData.index,inplace=True)
####################################################################

In [9]:
######        Machine Learning Section    ###########




X = data.drop('comment_count',axis=1)
y = data[['comment_count']]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


#           GBT model and evaluation


##from sklearn.model_selection import cross_val_score

###rom sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
predictions = reg.predict(X_test)


#evaluation

from sklearn.metrics import mean_absolute_error,mean_squared_error

def evaluate():
    """
    Evaluation function
    
    No input
    
    Print model's evaluation by what's in 'predictions' variable
    """
    
    MAE = round(mean_absolute_error(predictions,y_test),5)
    
    print('\n\n')
    print('The RMSE is: ' , round(np.sqrt(mean_squared_error(predictions,y_test)),5))
    print('The MAE  is: ' , round(MAE,5))
    
    print('\n')
    print('The mean is: ', y.mean())
    print('The rational MAE is: ' , round(MAE/y.mean(),3))

C:\Users\97250\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [10]:
###          tree      #####

from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(random_state=0)
#cross_val_score(regressor, X_train, y_train, cv=10)

regressor.fit(X_train,y_train)

predictions = regressor.predict(X_test)



#evaluation

evaluate()

##tree achived 0.3
## With all data tree achived 0.14

#############################




The RMSE is:  20526.41803
The MAE  is:  3668.79493


The mean is:  comment_count    9531.851361
dtype: float64
The rational MAE is:  comment_count    0.385
dtype: float64


In [11]:
#           random forest model and evaluation


from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=600)

rfr.fit(X_train,y_train)

predictions = rfr.predict(X_test)

#evaluation

evaluate()

##not better than Tree model
##All data - 0.115 Damm

###############################################

<ipython-input-11-970326f27fc4>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr.fit(X_train,y_train)





The RMSE is:  15462.10686
The MAE  is:  3245.03022


The mean is:  comment_count    9531.851361
dtype: float64
The rational MAE is:  comment_count    0.34
dtype: float64


In [12]:
#       filter columns

#correlation matrix
correlated = data.corr()['comment_count']
#top 12 features
corrFeat = correlated.sort_values(ascending=False)

corrFeat =corrFeat[:12].index

data = data[corrFeat]

## worse outcomes 
## RFE had only 90 seconds to go of 600 trees
## 0.18 instead of 0.115
# so its realy good to add features like I did

#########################################################


############        final!!             #################


########################################################